In [ ]:
import os
import numpy as np
import random
import nltk 
from nltk.tokenize import word_tokenize
import json
from tqdm import tqdm

In [ ]:
import keras
from keras.utils.np_utils import *
from keras.utils.np_utils import to_categorical
from keras.layers import *
from keras.models import Model, load_model
from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers 
from keras.optimizers import *

In [ ]:
root_path = './'
UserLogName = 'UserLog.tsv'
NewsContentName = 'News.tsv'

In [ ]:
def read_news(path,filenames):
    news={}
    category=[]
    news_index={}
    index=1
    word_dict={}
    word_index=1
    with open(os.path.join(path,filenames), encoding='utf-8') as f:
        lines=f.readlines()
    for line in tqdm(lines):
        splited = line.strip('\n').split('\t')
        doc_id,vert,title,_= splited[0:4]
        news_index[doc_id]=index
        index+=1
        category.append(vert)
        title = title.lower()
        title=title.split()
        news[doc_id]=[vert,title]
        for word in title:
            word = word.lower()
            if not(word in word_dict):
                word_dict[word]=word_index
                word_index+=1
    category=list(set(category))
    category_dict={}
    index=0
    for c in category:
        category_dict[c]=index
        index+=1

    return news,news_index,category_dict,word_dict

In [152]:
with open(os.path.join(root_path,NewsContentName), encoding='utf-8') as f:
    lines=f.readlines()
    for line in lines[:10]:
        print(line.strip('\n').split('\t'))

['442A918544141879438307453F128295', 'tp_animals,hp1_science,tp_science,hp1_environment,tp_nature,tp_zoology,tp_evolutionary_biology,tp_animal_behavior,sg_science,tp8_giraffes,tp8_lightning,tp8_animals,tp8_weather,tp8_evolutionary_biology,tp8_entomology,tp8_zoology,tp8_species,tp8_nature,tp8_wild_life,sg8_environment,kw_lightning,expsg_science,hpsg_science,hpsg_environment,fasttp_animals,fasttp_giraffes,fasttp_weather,fasttp_zoology,kp_fuzzy_lightning_rods,kp_giraffes,kp_south_africa,kp_luca_galuzzi,kp_george_dvorsky,kp_basically_fuzzy_lightning_rods,kp_research', 'Giraffes Are Basically Fuzzy Lightning Rods, New Research Suggests', 'Earlier this year, conservationists in South Africa discovered two adult giraffes struck down by lightning, signa ling a potentially underrated risk for this majestic species. Lightning kills around 27 people in the United States each year, but the quantity and frequency of animal deaths as a result of these electrical outbursts is a complete mystery.']
['

In [ ]:
%time news,news_index,category_dict,word_dict = read_news(root_path,NewsContentName)

In [ ]:
MAX_SENTENCE = 30
def get_doc_input(news,news_index,category,word_dict):
    news_num=len(news)+1
    news_title=np.zeros((news_num,MAX_SENTENCE),dtype='int32')
    news_vert=np.zeros((news_num,),dtype='int32')
    for key in tqdm(news):    
        vert,title=news[key]
        doc_index=news_index[key]
        news_vert[doc_index]=category[vert]
        for word_id in range(min(MAX_SENTENCE,len(title))):
            news_title[doc_index,word_id]=word_dict[title[word_id].lower()]
        
    return news_title,news_vert

In [ ]:
news_title,news_vert=get_doc_input(news,news_index,category_dict,word_dict)

In [154]:
with open(os.path.join(root_path,UserLogName)) as f:
    lines = f.readlines()
    for line in lines[:20]:
        print(line.strip('\n').split('\t'))

['0000345EFA9B41A88AEC4DFEF5EBED40', 'FFFFF', '1601141277', 'scroll', 'Opal_VideoCandidate_0916&Android', '9ACF01F02C244E629E16D384D9481F0B', '', '', '0', '0']
['0000345EFA9B41A88AEC4DFEF5EBED40', 'A7362C72BCA7376007AA7517BBD56226', '1601141277', 'seen', 'Opal_VideoCandidate_0916&Android', '9ACF01F02C244E629E16D384D9481F0B', 'NewsDigest', '0', '44.449', '0']
['0000345EFA9B41A88AEC4DFEF5EBED40', 'E09E1EB34E63D00848F400D83242E0D6', '1601141277', 'seen', 'Opal_VideoCandidate_0916&Android', '9ACF01F02C244E629E16D384D9481F0B', 'NewsDigest', '1', '0', '0']
['0000345EFA9B41A88AEC4DFEF5EBED40', 'FFFFF', '1601209489', 'scroll', 'Opal&Android', '151353D1D2E347349BD9ADA00FF0B8F5', '', '', '0', '0']
['0000345EFA9B41A88AEC4DFEF5EBED40', '95D02CD332A88D200C752D6915017227', '1601209489', 'seen', 'Opal&Android', '151353D1D2E347349BD9ADA00FF0B8F5', 'MiniCard', '4', '0', '0']
['0000345EFA9B41A88AEC4DFEF5EBED40', 'B509CAD0C0CE5DBBDD07A4F55AA695C0', '1601209489', 'seen', 'Opal&Android', '151353D1D2E347349

In [ ]:
class Logs():
    def __init__(self,newsid,eventime,action,dt):
        self.newsid = newsid
        self.eventime = eventime
        self.action = action
        self.dt = dt
    def __lt__(self,log):
        return self.eventime<log.eventime
    def __gt__(self,log):
        return self.eventime>log.eventime
    def __eq__(self,log):
        return self.eventime==log.eventime

In [ ]:
actiontypeid={'NULL': -1,
 'scroll':-1,
 'seen': 0,
 'click': 1,
 'articlestatus': 0,
 'read': 2,
 'play': 0,
 'pause': 0,
 'finish': 3,
 'share': 4,
 'pulltorefresh': -1,
 'dislikeprovider': 5,
 'dislikecard': 5,
 'report': -1,
 'saved': -1,
 'fullscreen': -1,
 'new_user': -1,
 'unseen': -1}

In [ ]:
import random
class User():
    def __init__(self,userid):
        self.userid = userid
        self.logs = []
        self.sort_flag = True
        
    def add_log(self,newsid,eventime,action,dt):
        eventime = int(eventime)
        self.logs.append(Logs(newsid,eventime,action,dt))
        self.sort_flag = False
    def sort_log(self,):
        self.logs.sort()
        self.sort_flag = True
    
    def positive_log(self,):
        if not self.sort_flag:
            self.sort_log()
            
        self.click_newsid = []
        self.click_action = []
        self.click_eventime = []
        self.click_dt = []
        for i in range(len(self.logs)):
            if actiontypeid[self.logs[i].action]!=-1:
                #if not self.logs[i].newsid in self.click_newsid:
                self.click_newsid.append(self.logs[i].newsid)
                self.click_eventime.append(self.logs[i].eventime)
                self.click_action.append(self.logs[i].action)
                self.click_dt.append(self.logs[i].dt)
                
        self.click_newsid = np.array(self.click_newsid)
        self.click_eventime = np.array(self.click_eventime)
        self.click_action = np.array(self.click_action)
        self.click_dt = np.array(self.click_dt)
        
    def get_impression(self,):
        self.impressions= []
        self.valid_impressions = []
        impression = []
        pos_flag = False
        neg_flag = False
        for i in range(len(self.logs)):
            log = self.logs[i]
            if log.action == 'scroll' or log.action == 'pulltorefresh':
                if len(impression)>0:
                    if pos_flag and neg_flag:
                        self.valid_impressions.append(impression)
                    self.impressions.append(impression)
                impression = []
                pos_flag = False
                neg_flag = False
                continue
            impression.append(i)
            if actiontypeid[log.action] in [1,2,3,4]:
                pos_flag = True
            if actiontypeid[log.action] in [0,5]:
                neg_flag = True
                
    def process_log_pos(self,):
        pass 

In [ ]:
Users = {}
skip = 0
skiped_news = {}
skiped_action={}
dt_dict = {}

for i in tqdm(range(len(lines))):
    userid, newsid, eventime, action, _,sid, size, position, lingertime, dt = lines[i].strip('\n').split('\t')
    if not userid in Users:
        Users[userid] = User(userid)
    if not dt in dt_dict:
        dt_dict[dt] = len(dt_dict) + 1
        
    eventime = int(eventime)
    if not newsid in news_index and newsid !='FFFFF':
        skip += 1
        if not newsid in skiped_news:
            skiped_news[newsid] = True
        skiped_action[action]=skiped_action.get(action,0)+1

        continue
    Users[userid].add_log(newsid,eventime,action,dt)

In [ ]:
for userid in tqdm(Users):
    Users[userid].sort_log()
    Users[userid].positive_log()
    Users[userid].get_impression()

In [ ]:
from datetime import datetime
import time
def trans2tsp(timestr):
    return int(time.mktime(datetime.strptime(timestr, '%m/%d/%Y %I:%M:%S %p').timetuple()))

anchor = trans2tsp('09/29/2020 07:00:00 AM')

In [ ]:
len(click_action)

In [ ]:
Samples = []
npratio=4
for cnt,userid in tqdm(enumerate(Users)):
    user = Users[userid]
    if cnt>=10:
        break
    logs = user.logs
    click_newsid = user.click_newsid
    click_action = user.click_action
    click_dt = user.click_dt
    if len(click_newsid) < 3:
        continue
    
    click_eventime = user.click_eventime
    imps = user.valid_impressions

    for imp in imps:
        
        imp_time = logs[imp[0]].eventime
        if imp_time>anchor:
            continue
            
        index = click_eventime<imp_time

        clicks = click_newsid[index]
        clicks = clicks.tolist()

        eventime = click_eventime[index]
        eventime = eventime.tolist()
        actions = click_action[index]
        actions = actions.tolist()
        dts = click_dt[index]
        dts = dts.tolist()
        
        reserveid=[]
        for ac in range(len(clicks)):
            if actiontypeid[actions[ac]]==0 and random.randint(0,10)!=0:    
                continue
            else:
                reserveid.append(ac)
        clicks=[clicks[y] for y in reserveid]
        actions=[actions[y] for y in reserveid]
        dts=[dts[y] for y in reserveid]
        et=[eventime[y] for y in reserveid]
        if len(et)==0:
            interval=[]
        else:
            interval=  [0]+[et[_+1]-et[_] for _ in range(len(et)-1)]  
        pos = []
        neg = []
        posaction = []
        negaction = []
        posdt = []
        negdt = []
        for logid in imp:
            log = logs[logid]
            if log.newsid in pos:
                continue
            if actiontypeid[log.action] in [1,2,3,4]:
                pos.append(log.newsid)
                posaction.append(log.action)
                posdt.append(log.dt)
            else:
                neg.append(log.newsid)
                negaction.append(log.action)
                negdt.append(log.dt)
        
        if len(neg) == 0 or len(pos) == 0:
            continue
                    
        #print(pos)
        #print(neg)
        
        index1 = np.random.permutation(len(pos))
        index2 = np.random.permutation(len(neg)) 
        
        if len(neg) < npratio:
            neg = neg * (npratio // len(neg) + 1)
            negaction = negaction * (npratio // len(negaction) + 1)
            negdt = negdt * (npratio // len(negdt) + 1)
        
        for i in range(len(pos)):
            index2 = np.random.permutation(len(neg))[:npratio]
            Samples.append([
                pos[index1[i]],posaction[index1[i]],posdt[index1[i]],
                [neg[i] for i in index2],[negaction[i] for i in index2],[negdt[i] for i in index2],
                imp_time,clicks[-100:],actions[-100:],dts[-100:],interval[-100:]])


In [ ]:
TestSamples = []

for cnt,userid in tqdm(enumerate(Users)):
    user = Users[userid]
    if cnt>=10:
        break
    logs = user.logs
    click_newsid = user.click_newsid
    click_action = user.click_action
    click_dt = user.click_dt
    if len(click_newsid) < 3:
        continue
        
    click_eventime = user.click_eventime 
    
    imps = user.valid_impressions

    for imp in imps:
        
        imp_time = logs[imp[0]].eventime
        if imp_time<=anchor:
            continue
            
        index = click_eventime<imp_time

        clicks = click_newsid[index]
        clicks = clicks.tolist()

        eventime = click_eventime[index]
        eventime = eventime.tolist() 
        actions = click_action[index]
        actions = actions.tolist()
        dts = click_dt[index]
        dts = dts.tolist()
        
        reserveid=[]
        for ac in range(len(clicks)):
            if actiontypeid[actions[ac]]==0 and random.randint(0,10)!=0:    
                continue
            else:
                reserveid.append(ac)
        clicks=[clicks[y] for y in reserveid]
        actions=[actions[y] for y in reserveid]
        dts=[dts[y] for y in reserveid]
        et=[eventime[y] for y in reserveid]
        if len(et)==0:
            interval=[]
        else:
            interval=  [0]+[et[_+1]-et[_] for _ in range(len(et)-1)]  
        posdt = []
        negdt = []
        pos = []
        neg = [] 
        posaction = []
        negaction = []
        posdt = []
        negdt = []
        for logid in imp:
            log = logs[logid]
            if log.newsid in pos:
                continue
            if actiontypeid[log.action] in [1,2,3,4]:
                pos.append(log.newsid)
                posaction.append(log.action)
                posdt.append(log.dt)
            else:
                neg.append(log.newsid)
                negaction.append(log.action)
                negdt.append(log.dt)
        
        if len(pos) ==0 or len(neg) ==0:
            continue

        TestSamples.append([pos,posaction,posdt,neg,negaction,negdt, imp_time,clicks[-100:],actions[-100:],dts[-100:],interval[-100:]])


In [ ]:
with open('News.tsv',encoding='utf-8') as f:
    newsdata=f.readlines()  
news={}
for line in newsdata:
    linesplit=line.strip().split('\t')
    news[linesplit[0]]=[word_tokenize(linesplit[2].lower())]
    
newsindex={'NULL':0} 
for newsid in news:
    newsindex[newsid]=len(newsindex) 
     

In [ ]:
def newsample(array,ratio):
    if ratio >len(array):
        return random.sample(array*(ratio//len(array)+1),ratio)
    else:
        return random.sample(array,ratio)

idx=0
npratio=4

train_candidate=[] 
train_candidate_dt=[] 
train_candidate_interval=[] 
train_candidate_action=[]  
train_candidate_mask=[] 
train_label=[] 
train_user_his=[]
train_user_hisaction=[]
train_user_hisdt=[]
train_user_hisinterval=[]
train_user_hismask0=[]
train_user_hismask1=[]
train_user_hismask2=[]
train_user_hismask3=[]
train_user_hismask4=[]
train_user_hismask5=[] 
for x in tqdm(Samples):
    
    if x[0] not in newsindex:
        continue
    clickids=[newsindex.get(k,0) for k in x[7]][-100:]    
    actionids=[actiontypeid.get(k,0) for k in x[8]][-100:] 
    clickdts=[np.round(np.log2(1+float(k))) for k in x[9]][-100:]
    clickintervals=[np.round(np.log2(1+float(k))) for k in x[10]][-100:]
    clickdtsraw= [float(k) for k in x[9][-100:]]
    actionids0=[int(k==0) for k in actionids]
    actionids1=[int(k==1) for k in actionids]
    
    for k in range(len(actionids)):
        if actionids[k]==2 and clickdtsraw[k]<10:
            actionids[k]=6
    
    actionids2=[int(k==3) for k in actionids]
    actionids3=[int(k==4) for k in actionids]
    actionids4=[int(k==5) for k in actionids]
    actionids5=[int(k==6) for k in actionids]
    pdoc = newsindex[x[0]]
    paction=actiontypeid.get(x[1],0)
    pdt = x[2]
    ndoc = [newsindex.get(k,0) for k in x[3]]
    naction=[actiontypeid.get(k,0) for k in x[4]]
    ndt = x[5]
    negd=ndoc+[pdoc]
    nega=naction+[paction]   
    negpdt=ndt+[pdt]    
    for k in range(len(negpdt)):
        if nega[k]==3:
            nega[k]=1
        else:
            nega[k]=0 

    negpdta=[np.round(np.log2(1+float(k))) for k in negpdt]
    
    candidate_label=[0]*npratio+[1]
    candidate_order=list(range(npratio+1))
    random.shuffle(candidate_order)
    candidate_shuffle=[]
    candidate_dt_shuffle=[]
    candidate_action_shuffle=[]
    candidate_label_shuffle=[]
    for i in candidate_order:
        candidate_shuffle.append(negd[i])
        candidate_action_shuffle.append(nega[i])
        candidate_label_shuffle.append(candidate_label[i])
        candidate_dt_shuffle.append(negpdta[i])
    train_candidate.append(candidate_shuffle)
    train_candidate_mask.append(np.sum(nega))
    train_candidate_dt.append(candidate_dt_shuffle)
    train_label.append(candidate_label_shuffle)
    train_user_his.append(clickids+[0]*(100-len(clickids))) 
    train_candidate_action.append(candidate_action_shuffle)
    train_user_hisdt.append(clickdts+[0]*(100-len(actionids))) 
    train_user_hisinterval.append(clickintervals+[0]*(100-len(actionids)))  
    print(len(clickintervals+[0]*(100-len(actionids))))
    train_user_hisaction.append(actionids+[0]*(100-len(actionids))) 
    train_user_hismask0.append(actionids0+[0]*(100-len(actionids0))) 
    train_user_hismask1.append(actionids1+[0]*(100-len(actionids1))) 
    train_user_hismask2.append(actionids2+[0]*(100-len(actionids2))) 
    train_user_hismask3.append(actionids3+[0]*(100-len(actionids3))) 
    train_user_hismask4.append(actionids4+[0]*(100-len(actionids4))) 
    train_user_hismask5.append(actionids5+[0]*(100-len(actionids5)))  

In [ ]:
test_candidate=[]  
test_candidate_action=[]    
test_candidate_dt=[]
test_label=[]
test_user_his=[]
test_user_hisdt=[]
test_user_hisinterval=[]
test_user_hisaction=[]
test_user_hismask0=[]
test_user_hismask1=[]
test_user_hismask2=[]
test_user_hismask3=[]
test_user_hismask4=[]
test_user_hismask5=[] 
test_index=[]
for x in tqdm(TestSamples):
    pdoc=[newsindex.get(k,0) for k in x[0] if k in newsindex]
    ndoc = [newsindex.get(k,0) for k in x[3] if k in newsindex]
    clickids=[newsindex.get(k,0) for k in x[7]][-100:]    
    actionids=[actiontypeid.get(k,0) for k in x[8]][-100:]     
    clickintervals=[np.round(np.log2(1+float(k))) for k in x[10]][-100:]
    clickdts=[np.round(np.log2(1+float(k))) for k in x[9]][-100:]
    clickdtsraw= [float(k) for k in x[9][-100:]]

    for k in range(len(actionids)):
        if actionids[k]==2 and clickdtsraw[k]<10:
            actionids[k]=6

    
    actionids0=[int(k==0) for k in actionids]
    actionids1=[int(k==1) for k in actionids]
    actionids2=[int(k==3) for k in actionids]
    actionids3=[int(k==4) for k in actionids]
    actionids4=[int(k==5) for k in actionids]
    actionids5=[int(k==6) for k in actionids]
    if len(pdoc)==0 or len(ndoc)==0:
        continue
        
    paction=[actiontypeid.get(k,0) for k in x[1]]
    pdt = [float(k) for k in x[2]]
    ndoc = [newsindex.get(k,0) for k in x[3]]
    naction=[actiontypeid.get(k,0) for k in x[4]]
    ndt = [float(k) for k in x[5]]
 

    index=[]
    index.append(len(test_candidate))
    for doc in range(len(pdoc)):
        test_candidate.append(pdoc[doc])
        test_candidate_action.append(paction[doc])
        test_candidate_dt.append(pdt[doc])
        test_label.append(1)
        test_user_hisdt.append(clickdts+[0]*(100-len(actionids))) 
        test_user_hisinterval.append(clickintervals+[0]*(100-len(actionids)))  
        test_user_his.append(clickids+[0]*(100-len(clickids)))  
        test_user_hisaction.append(actionids+[0]*(100-len(actionids))) 
        test_user_hismask0.append(actionids0+[0]*(100-len(actionids0))) 
        test_user_hismask1.append(actionids1+[0]*(100-len(actionids1))) 
        test_user_hismask2.append(actionids2+[0]*(100-len(actionids2))) 
        test_user_hismask3.append(actionids3+[0]*(100-len(actionids3))) 
        test_user_hismask4.append(actionids4+[0]*(100-len(actionids4))) 
        test_user_hismask5.append(actionids5+[0]*(100-len(actionids5)))  
    for doc in range(len(ndoc)):
        test_candidate.append(ndoc[doc])
        test_candidate_action.append(naction[doc])
        test_candidate_dt.append(ndt[doc])
        test_label.append(0)
        test_user_hisdt.append(clickdts+[0]*(100-len(actionids))) 
        test_user_hisinterval.append(clickintervals+[0]*(100-len(actionids)))  
        test_user_his.append(clickids+[0]*(100-len(clickids))) 
        test_user_hisaction.append(actionids+[0]*(100-len(actionids))) 
        test_user_hismask0.append(actionids0+[0]*(100-len(actionids0))) 
        test_user_hismask1.append(actionids1+[0]*(100-len(actionids1))) 
        test_user_hismask2.append(actionids2+[0]*(100-len(actionids2))) 
        test_user_hismask3.append(actionids3+[0]*(100-len(actionids3))) 
        test_user_hismask4.append(actionids4+[0]*(100-len(actionids4))) 
        test_user_hismask5.append(actionids5+[0]*(100-len(actionids5)))  
    index.append(len(test_candidate))
    test_index.append(index)

In [ ]:
word_dict={'PADDING':[0,999999]}

for newsid in news:
    title=[]
    for word in news[newsid][0]:
        if word not in word_dict:
            word_dict[word]=[len(word_dict),1]
        else:
            word_dict[word][1]+=1


In [ ]:
word_dict1={}
for i in word_dict:
    if word_dict[i][1]>=2 and i not in word_dict1:
        word_dict1[i]=[len(word_dict1),word_dict[i][1]]
print(len(word_dict1))

In [ ]:

news_title=[[0]*30]

for newsid in news:
    title=[]
    for word in news[newsid][0]:
        if word in word_dict1:
            title.append(word_dict1[word][0])
    title=title[:30]
    news_title.append(title+[0]*(30-len(title)))

news_title=np.array(news_title,dtype='int32') 

In [ ]:
train_candidate_dt=np.array(train_candidate_dt,dtype='float32')
train_candidate=np.array(train_candidate,dtype='int32')
train_candidate_mask=np.array(train_candidate_mask,dtype='float32')
train_candidate_action=np.array(train_candidate_action,dtype='int32')
train_label=np.array(train_label,dtype='int32')
train_user_his=np.array(train_user_his,dtype='int32')
train_user_hisaction=np.array(train_user_hisaction,dtype='int32')
train_user_hisinterval=np.array(train_user_hisinterval,dtype='int32')
train_user_hisdt=np.array(train_user_hisdt,dtype='int32')
train_user_hismask0=np.array(train_user_hismask0,dtype='int32')
train_user_hismask1=np.array(train_user_hismask1,dtype='int32')
train_user_hismask2=np.array(train_user_hismask2,dtype='int32')
train_user_hismask3=np.array(train_user_hismask3,dtype='int32')
train_user_hismask4=np.array(train_user_hismask4,dtype='int32')
train_user_hismask5=np.array(train_user_hismask5,dtype='int32') 

In [ ]:

test_candidate=np.array(test_candidate,dtype='int32') 
test_candidate_action=np.array(test_candidate_action,dtype='int32') 
test_candidate_dt=np.array(test_candidate_dt,dtype='float32') 
test_user_his=np.array(test_user_his,dtype='int32')
test_label=np.array(test_label,dtype='int32')
test_user_hisaction=np.array(test_user_hisaction,dtype='int32')
test_user_hisinterval=np.array(test_user_hisinterval,dtype='int32')
test_user_hisdt=np.array(test_user_hisdt,dtype='int32')
test_user_hismask0=np.array(test_user_hismask0,dtype='int32')
test_user_hismask1=np.array(test_user_hismask1,dtype='int32')
test_user_hismask2=np.array(test_user_hismask2,dtype='int32')
test_user_hismask3=np.array(test_user_hismask3,dtype='int32')
test_user_hismask4=np.array(test_user_hismask4,dtype='int32')
test_user_hismask5=np.array(test_user_hismask5,dtype='int32') 

In [ ]:

def generate_batch_data_random(batch_size):
    idlist = np.arange(len(train_label))
    np.random.shuffle(idlist)
    y=train_label
    batches = [idlist[range(batch_size*i, min(len(y), batch_size*(i+1)))] for i in range(len(y)//batch_size+1)]
    while (True):
        for i in batches:
            item = news_title[train_candidate[i]]
            user = news_title[train_user_his[i]] 
            useraction = train_user_hisaction[i]
            userinterval = train_user_hisinterval[i]
            userdt = train_user_hisdt[i]
            itemaction= train_candidate_action[i]
            useractionmask0=train_user_hismask0[i]
            useractionmask1=train_user_hismask1[i]
            useractionmask2=train_user_hismask2[i]
            useractionmask3=train_user_hismask3[i]
            useractionmask4=train_user_hismask4[i]
            useractionmask5=train_user_hismask5[i] 
            itemdt=np.sum(train_candidate_dt[i],axis=-1,keepdims=True)
            yield ([item,user,useraction,userdt,userinterval,useractionmask0,useractionmask1,
                    useractionmask2,useractionmask3,useractionmask4,useractionmask5],
                   [y[i],itemdt,itemaction,np.zeros((batch_size,1))],[np.ones((batch_size,)),np.ones((batch_size,)),train_candidate_mask[i]+1e-8,np.ones((batch_size,))])


def generate_batch_data(batch_size):
    idlist = np.arange(len(test_candidate))
    batches = [idlist[range(batch_size*i, min(len(idlist), batch_size*(i+1)))] for i in range(len(idlist)//batch_size+1)]

    while (True):
        for i in batches:
            item = news_title[test_candidate[i]]
            user=news_title[test_user_his[i]]
            useraction = test_user_hisaction[i]
            userinterval = test_user_hisinterval[i]
            userdt = test_user_hisdt[i]
            useractionmask0=test_user_hismask0[i]
            useractionmask1=test_user_hismask1[i]
            useractionmask2=test_user_hismask2[i]
            useractionmask3=test_user_hismask3[i]
            useractionmask4=test_user_hismask4[i]
            useractionmask5=test_user_hismask5[i] 
            yield ([item,user,useraction,userdt,userinterval,useractionmask0,useractionmask1,useractionmask2,useractionmask3,useractionmask4,useractionmask5])
            

In [ ]:
import numpy as np
embdict={}
import pickle
with open('glove.840B.300d.txt','rb')as f:
    while True:
        j=f.readline()
        if len(j)==0:
            break
        k = j.split()
        word=k[0].decode()
        if len(word) != 0:
            tp=[float(x) for x in k[1:]]
            if word in word_dict1:
                embdict[word]=tp



In [ ]:
from numpy.linalg import cholesky

emb_table=[0]*len(word_dict1)
xp=np.zeros(300,dtype='float32')

cand=[]
for i in embdict:
    emb_table[word_dict1[i][0]]=np.array(embdict[i],dtype='float32')
    cand.append(emb_table[word_dict1[i][0]])
cand=np.array(cand,dtype='float32')

mu=np.mean(cand, axis=0)
Sigma=np.cov(cand.T)

norm=np.random.multivariate_normal(mu, Sigma, 1)
print(mu.shape,Sigma.shape,norm.shape)

for i in range(len(emb_table)):
    if type(emb_table[i])==int:
        emb_table[i]=np.reshape(norm, 300)
emb_table[0]=np.zeros(300,dtype='float32')
emb_table=np.array(emb_table,dtype='float32')
print(emb_table.shape)


In [ ]:
import keras
from keras.layers import *
from keras.models import Model
from keras import backend as K
from keras.optimizers import *
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [ ]:

def dcg_score(y_true, y_score, k=10):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])
    gains = 2 ** y_true - 1
    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gains / discounts)


def ndcg_score(y_true, y_score, k=10):
    best = dcg_score(y_true, y_true, k)
    actual = dcg_score(y_true, y_score, k)
    return actual / best


def mrr_score(y_true, y_score):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order)
    rr_score = y_true / (np.arange(len(y_true)) + 1)
    return np.sum(rr_score) / np.sum(y_true)


In [ ]:

class Attention(Layer):

    def __init__(self, nb_head, size_per_head, **kwargs):
        self.nb_head = nb_head
        self.size_per_head = size_per_head
        self.output_dim = nb_head*size_per_head
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.WQ = self.add_weight(name='WQ', 
                                  shape=(input_shape[0][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        self.WK = self.add_weight(name='WK', 
                                  shape=(input_shape[1][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        self.WV = self.add_weight(name='WV', 
                                  shape=(input_shape[2][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        super(Attention, self).build(input_shape)
        
    def Mask(self, inputs, mask, mode='mul'):
        if mask == None:
            return inputs
        else:
            
            
            if mode == 'mul':
                mask = K.expand_dims(mask,axis=2)
                return inputs * mask
            if mode == 'add':
                mask = tf.matmul(K.expand_dims(mask, axis=-1),K.expand_dims(mask, axis=1))
                mask = K.expand_dims(mask,axis=3)
                return inputs - (1 - mask) * 1e12
                
    def call(self, x):
        if len(x) == 3:
            Q_seq,K_seq,V_seq = x
            mask = None
        elif len(x) == 4:
            Q_seq,K_seq,V_seq,mask = x
        Q_seq = K.dot(Q_seq, self.WQ)
        Q_seq = K.reshape(Q_seq, (-1, K.shape(Q_seq)[1], self.nb_head, self.size_per_head))
        Q_seq = K.permute_dimensions(Q_seq, (0,2,1,3))
        K_seq = K.dot(K_seq, self.WK)
        K_seq = K.reshape(K_seq, (-1, K.shape(K_seq)[1], self.nb_head, self.size_per_head))
        K_seq = K.permute_dimensions(K_seq, (0,2,1,3))
        V_seq = K.dot(V_seq, self.WV)
        V_seq = K.reshape(V_seq, (-1, K.shape(V_seq)[1], self.nb_head, self.size_per_head))
        V_seq = K.permute_dimensions(V_seq, (0,2,1,3))
        A = K.batch_dot(Q_seq, K_seq, axes=[3,3]) / self.size_per_head**0.5
        A = K.permute_dimensions(A, (0,3,2,1))

        A = self.Mask(A, mask, 'add')
        A = K.permute_dimensions(A, (0,3,2,1))    
        A = K.softmax(A)
        O_seq = K.batch_dot(A, V_seq, axes=[3,2])
        O_seq = K.permute_dimensions(O_seq, (0,2,1,3))
        O_seq = K.reshape(O_seq, (-1, K.shape(O_seq)[1], self.output_dim))
        O_seq = self.Mask(O_seq, mask, 'mul')
        return O_seq
        
    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], input_shape[0][1], self.output_dim)

In [ ]:
MAX_SENT_LENGTH=30
MAX_SENTS=100


keras.backend.clear_session() 
sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')

embedding_layer = Embedding(len(word_dict1), weights=[emb_table],300,trainable=True)
 
embedded_sequences = embedding_layer(sentence_input)

d_et=Dropout(0.2)(embedded_sequences)

d_ct= Attention(16,16)([d_et,d_et,d_et])
#l_att=Dense(64,activation='tanh')(l_att)
attention = Dense(200,activation='tanh')(d_ct)
attention = Flatten()(Dense(1)(attention))
attention_weight = Activation('softmax')(attention)
l_att=keras.layers.Dot((1, 1))([d_ct, attention_weight])


sentEncodert = Model([sentence_input], l_att)


news_input = Input((MAX_SENTS,MAX_SENT_LENGTH,), dtype='int32')
news_encoders= TimeDistributed(sentEncodert)(news_input)

newsaction_input = Input((MAX_SENTS,), dtype='int32')
action_embedding_layer = Embedding(7, 256,trainable=True)
action_embedded_sequences = action_embedding_layer(newsaction_input)

newsdt_input = Input((MAX_SENTS,), dtype='int32')
dt_embedding_layer = Embedding(32, 256,trainable=True)
dt_embedded_sequences = dt_embedding_layer(newsdt_input)

newsinterval_input = Input((MAX_SENTS,), dtype='int32')
interval_embedding_layer = Embedding(32, 256,trainable=True)
interval_embedded_sequences = interval_embedding_layer(newsinterval_input)

position_embedding_layer = Embedding(100, 256,trainable=True)
position_embedded_sequences = position_embedding_layer(Lambda(lambda x: K.arange(K.int_shape(x)[-1])+K.zeros_like(x,dtype='int32'))(newsinterval_input))

news_encoders = add([news_encoders,action_embedded_sequences,dt_embedded_sequences,interval_embedded_sequences,position_embedded_sequences])
u_att = Attention(16,16)([news_encoders,news_encoders,news_encoders])


newsactionmask0 = Input((MAX_SENTS,), dtype='float32')
newsactionmask1 = Input((MAX_SENTS,), dtype='float32')
newsactionmask2 = Input((MAX_SENTS,), dtype='float32')
newsactionmask3 = Input((MAX_SENTS,), dtype='float32')
newsactionmask4 = Input((MAX_SENTS,), dtype='float32')
newsactionmask5 = Input((MAX_SENTS,), dtype='float32') 

u_att0_rep = Attention(16,16)([u_att,u_att,u_att,newsactionmask0])
u_att1_rep = Attention(16,16)([u_att,u_att,u_att,newsactionmask1])
u_att2_rep = Attention(16,16)([u_att,u_att,u_att,newsactionmask2])
u_att3_rep = Attention(16,16)([u_att,u_att,u_att,newsactionmask3])
u_att4_rep = Attention(16,16)([u_att,u_att,u_att,newsactionmask4])
u_att5_rep = Attention(16,16)([u_att,u_att,u_att,newsactionmask5])
u_att3_att=Flatten()(Dense(1)(u_att3_rep))
u_att4_att=Flatten()(Dense(1)(u_att4_rep))

mask_pn=add([newsactionmask0,newsactionmask1])

u_att_strong_exp_pos=Dot((1, 1))([u_att3_rep,Activation('softmax')(Lambda (lambda x:x[0]-(1-x[1])*99)([u_att3_att,newsactionmask3]))]) 
u_att_strong_exp_neg=Dot((1, 1))([u_att4_rep,Activation('softmax')(Lambda (lambda x:x[0]-(1-x[1])*99)([u_att4_att,newsactionmask4]))])

u_att2_att=Dot((2, 1))([u_att2_rep,u_att_strong_exp_pos])
u_att5_att=Dot((2, 1))([u_att5_rep,u_att_strong_exp_neg])
u_att_strong_imp_pos=Dot((1, 1))([u_att2_rep,Activation('softmax')(Lambda (lambda x:x[0]-(1-x[1])*99)([u_att2_att,newsactionmask2]))]) 
u_att_strong_imp_neg=Dot((1, 1))([u_att5_rep,Activation('softmax')(Lambda (lambda x:x[0]-(1-x[1])*99)([u_att5_att,newsactionmask5]))])

u_att_strong_pos=add([u_att_strong_exp_pos,u_att_strong_imp_pos])
u_att_strong_neg=add([u_att_strong_exp_neg,u_att_strong_imp_neg])

pn_rep = Lambda (lambda x:x[0]*K.expand_dims(x[2],axis=2)+x[1]*K.expand_dims(x[3],axis=2))([u_att0_rep,u_att1_rep,newsactionmask0,newsactionmask1])

u_att0_att=Dot((2, 1))([pn_rep,u_att_strong_exp_pos])
u_att1_att=Dot((2, 1))([pn_rep,u_att_strong_exp_neg])
u_att_weak_imp_pos=Dot((1, 1))([pn_rep,Activation('softmax')(Lambda (lambda x:x[0]-(1-x[1])*99)([u_att0_att,mask_pn]))]) 
u_att_weak_imp_neg=Dot((1, 1))([pn_rep,Activation('softmax')(Lambda (lambda x:x[0]-(1-x[1])*99)([u_att1_att,mask_pn]))])

pndisentangle=Lambda(lambda x:K.abs(x))(Dot((1,1), normalize=True)([Lambda(lambda x:x+1e-8)(u_att_weak_imp_pos),Lambda(lambda x:x+1e-8)(u_att_weak_imp_neg)]))



posvec =concatenate([Lambda(lambda x: K.expand_dims(x,axis=1))(vec) for vec in [u_att_strong_exp_pos,u_att_strong_imp_pos]],axis=1)
attentionv= Activation('softmax')(Flatten()(Dense(1)(Dense(200,activation='tanh')(posvec))))
u_strong_pos=keras.layers.Dot((1, 1))([posvec, attentionv])

negvec =concatenate([Lambda(lambda x: K.expand_dims(x,axis=1))(vec) for vec in [u_att_strong_exp_neg,u_att_strong_imp_neg]],axis=1)
attentionv2= Activation('softmax')(Flatten()(Dense(1)(Dense(200,activation='tanh')(negvec))))
u_strong_neg=keras.layers.Dot((1, 1))([negvec, attentionv2])

allvec =concatenate([Lambda(lambda x: K.expand_dims(x,axis=1))(vec) for vec in [u_strong_pos,u_strong_neg,u_att_weak_imp_pos,u_att_weak_imp_neg]],axis=1)
attention_final= Activation('softmax')(Flatten()(Dense(1)(allvec)))
uemb=keras.layers.Dot((1, 1))([allvec, attention_final])

#u_att = Dot((1, 1))([news_encoders,Activation('softmax')(Flatten()(Dense(1)(Dense(200,activation='tanh')(news_encoders ))))])

candidates=Input((1+npratio,MAX_SENT_LENGTH,), dtype='int32')
candidate_vecs=TimeDistributed(sentEncodert)(candidates)

logits = Activation('softmax')(keras.layers.dot([candidate_vecs, uemb], axes=-1)) 
pred_dense1=Dense(256)(candidate_vecs)
pred_dense2=Dense(256)(candidate_vecs)

finish = Activation('softmax')(keras.layers.dot([pred_dense1, uemb], axes=-1))
dwelltime = Lambda(lambda x:K.sum(x,axis=-1,keepdims=True))(Activation('relu')(keras.layers.dot([pred_dense2, uemb], axes=-1)))

model = Model([candidates,news_input,newsaction_input,newsdt_input,newsinterval_input,newsactionmask0,newsactionmask1,
               newsactionmask2,newsactionmask3,newsactionmask4,newsactionmask5], [logits,dwelltime,finish,pndisentangle])

model.compile(loss=['categorical_crossentropy','mae','categorical_crossentropy','mae'], optimizer=Adam(lr=0.0001),metrics=['acc'],loss_weights=[1.,0.15,0.3,0.2])


candidate_one = keras.Input((MAX_SENT_LENGTH,)) 

candidate_one_vec = sentEncodert([candidate_one]) 

score = keras.layers.dot([candidate_one_vec, uemb], axes=-1) 

model2 = keras.Model([candidate_one,news_input,newsaction_input,newsdt_input,newsinterval_input,newsactionmask0,newsactionmask1,
               newsactionmask2,newsactionmask3,newsactionmask4,newsactionmask5], score)

In [ ]:
from sklearn.metrics import roc_auc_score

from keras.optimizers import *
results=[]
all_pred=[]
for ep in range(1):
    traingen=generate_batch_data_random(50)
    model.fit_generator(traingen, epochs=1,steps_per_epoch=len(train_label)//50)
    valgen=generate_batch_data(50)
    ider=0
    cr = model2.predict_generator(valgen, steps=len(test_label)//50,verbose=1)
    from sklearn.metrics import roc_auc_score
    all_auc=[]
    all_mrr=[]
    all_ndcg=[]
    all_ndcg2=[]
    for m in test_index:
        if np.sum(test_label[m[0]:m[1]])!=0 and m[1]<len(cr):
    
            all_auc.append(roc_auc_score(test_label[m[0]:m[1]],cr[m[0]:m[1],0]))
            all_mrr.append(mrr_score(test_label[m[0]:m[1]],cr[m[0]:m[1],0]))
            all_ndcg.append(ndcg_score(test_label[m[0]:m[1]],cr[m[0]:m[1],0],k=5))
            all_ndcg2.append(ndcg_score(test_label[m[0]:m[1]],cr[m[0]:m[1],0],k=10))
    print(np.mean(all_auc),np.mean(all_mrr),np.mean(all_ndcg),np.mean(all_ndcg2))
    
    allf=0
    topf=0
    topdt=0
    alls=0
    alld=0
    
    clicks=0
    alldtr=0
    alldtc=0
    for m in test_index:
        if np.sum(test_label[m[0]:m[1]])!=0 and m[1]<len(cr):
            dd=np.argsort(cr[m[0]:m[1],0])[::-1][:5]
            clicks+=np.sum(test_label[m[0]:m[1]][dd])
            topdt+=np.mean(test_candidate_dt[m[0]:m[1]][dd])
            for j in test_candidate_action[m[0]:m[1]]:
                if j==3:
                    allf+=1
                
            for j in dd:
                if test_candidate_action[m[0]:m[1]][j]==3:
                    topf+=1
                    
                if test_candidate_action[m[0]:m[1]][j]==4:
                    alls+=1
                if test_candidate_action[m[0]:m[1]][j]==5:
                    alld+=1
                    
                if test_candidate_action[m[0]:m[1]][j]==2:
                    alldtc+=1
                    alldtr+=test_candidate_dt[m[0]:m[1]][j]
                    
    print(allf,topf,topdt/len(test_index),alls/len(test_index)/5,alld/len(test_index)/5)
    results.append([np.mean(all_auc),np.mean(all_mrr),np.mean(all_ndcg),np.mean(all_ndcg2),clicks/5/len(test_index),alldtr/alldtc,alls/len(test_index)/5,alld/len(test_index)/5])

In [ ]:
results

In [ ]:
results

In [ ]:
clicks=0
for m in test_index:
    if np.sum(test_label[m[0]:m[1]])!=0 and m[1]<len(cr):
        dd=np.argsort(cr[m[0]:m[1],0])[::-1][:5]
        #print(dd)
        clicks+=np.sum(test_label[m[0]:m[1]][dd])
print(clicks/5/len(test_index))


In [ ]:
(0.00012239-9.704349e-5)/9.704349e-5

In [ ]:
0.0003168659698026204+0.0002825636925176165

In [ ]:
0.0005396/0.0005994296623202369

In [ ]:
results

In [ ]:
allf=0
topf=0
topdt=0
alls=0
alld=0
for m in test_index:
    if np.sum(test_label[m[0]:m[1]])!=0 and m[1]<len(cr):
        dd=np.argsort(cr[m[0]:m[1],0])[::-1][:5]
        #print(dd)
        topdt+=np.mean(test_candidate_dt[m[0]:m[1]][dd])
        for j in test_candidate_action[m[0]:m[1]]:
            if j==3:
                allf+=1
            
        for j in dd:
            if test_candidate_action[m[0]:m[1]][j]==3:
                topf+=1
                
            if test_candidate_action[m[0]:m[1]][j]==4:
                alls+=1
            if test_candidate_action[m[0]:m[1]][j]==5:
                alld+=1
print(allf,topf,topdt/len(test_index),alls/len(test_index)/5,alld/len(test_index)/5)
    


In [ ]:
allf=0
topf=0
topdt=0
alls=0
alld=0
for m in test_index:
    if np.sum(test_label[m[0]:m[1]])!=0 and m[1]<len(cr):
        dd=np.argsort(cr[m[0]:m[1],0])[::-1][:5]
        #print(dd)
        topdt+=np.mean(test_candidate_dt[m[0]:m[1]][dd])
        for j in test_candidate_action[m[0]:m[1]]:
            if j==3:
                allf+=1
            
        for j in dd:
            if test_candidate_action[m[0]:m[1]][j]==3:
                topf+=1
                
            if test_candidate_action[m[0]:m[1]][j]==4:
                alls+=1
            if test_candidate_action[m[0]:m[1]][j]==5:
                alld+=1
print(allf,topf,topdt/len(test_index),alls/len(test_index)/5,alld/len(test_index)/5)
    


In [ ]:
allf=0
topf=0
topdt=0
alls=0
alld=0
for m in test_index:
    if np.sum(test_label[m[0]:m[1]])!=0 and m[1]<len(cr):
        dd=np.argsort(cr[m[0]:m[1],0])[::-1][:5]
        #print(dd)
        topdt+=np.mean(test_candidate_dt[m[0]:m[1]][dd])
        for j in test_candidate_action[m[0]:m[1]]:
            if j==3:
                allf+=1
            
        for j in dd:
            if test_candidate_action[m[0]:m[1]][j]==3:
                topf+=1
                
            if test_candidate_action[m[0]:m[1]][j]==4:
                alls+=1
            if test_candidate_action[m[0]:m[1]][j]==5:
                alld+=1
print(allf,topf,topdt/len(test_index),alls/len(test_index)/5,alld/len(test_index)/5)
    


In [ ]:
share 9.704349479606015e-05
0.0003168659698026204+0.0002825636925176165

In [ ]:
allf=0
topf=0
topdt=0
for m in test_index:
    if np.sum(test_label[m[0]:m[1]])!=0 and m[1]<len(cr):
        dd=np.argsort(cr[m[0]:m[1],0])[::-1][:5]
        #print(dd)
        topdt+=np.mean(test_candidate_dt[m[0]:m[1]][dd])
        for j in test_candidate_action[m[0]:m[1]]:
            if j==3:
                allf+=1
        for j in dd:
            if test_candidate_action[m[0]:m[1]][j]==3:
                topf+=1
print(allf,topf,topdt/len(test_index))
    


In [ ]:
MAX_SENT_LENGTH=30
MAX_SENTS=200

loss_action_weight={0:1.,1:1.,2:1.,3:2.,4:3.,5:3.,6:1.,-1:0.}
def generate_batch_data_random(batch_size):
    idlist = np.arange(len(train_label))
    np.random.shuffle(idlist)
    y=train_label
    batches = [idlist[range(batch_size*i, min(len(y), batch_size*(i+1)))] for i in range(len(y)//batch_size+1)]
    while (True):
        for i in batches:
            item = news_title[train_candidate[i]]
            user = news_title[train_user_his[i]] 
            useraction = train_user_hisaction[i]
            itemaction= train_candidate_action[i]
            useractionmask0=train_user_hismask0[i]
            useractionmask1=train_user_hismask1[i]
            useractionmask2=train_user_hismask2[i]
            useractionmask3=train_user_hismask3[i]
            useractionmask4=train_user_hismask4[i]
            useractionmask5=train_user_hismask5[i]
            useractionmask6=train_user_hismask6[i]
            itemdt=train_candidate_dt[i]
            yield ([item,user,useraction,useractionmask0,useractionmask1,
                    useractionmask2,useractionmask3,useractionmask4,useractionmask5,useractionmask6], [y[i]])#,itemdt,to_categorical(itemaction,4)
            


def generate_batch_data(batch_size):
    idlist = np.arange(len(test_candidate))
    batches = [idlist[range(batch_size*i, min(len(idlist), batch_size*(i+1)))] for i in range(len(idlist)//batch_size+1)]

    while (True):
        for i in batches:
            item = news_title[test_candidate[i]]
            user=news_title[test_user_his[i]]
            useraction = test_user_hisaction[i]
            useractionmask0=test_user_hismask0[i]
            useractionmask1=test_user_hismask1[i]
            useractionmask2=test_user_hismask2[i]
            useractionmask3=test_user_hismask3[i]
            useractionmask4=test_user_hismask4[i]
            useractionmask5=test_user_hismask5[i]
            useractionmask6=test_user_hismask6[i]
            yield ([item,user,useraction,useractionmask0,useractionmask1,useractionmask2,useractionmask3,useractionmask4,useractionmask5,useractionmask6])
            


keras.backend.clear_session() 
sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')

embedding_layer = Embedding(len(word_dict1), 300, weights=[lister],trainable=True)
 
embedded_sequences = embedding_layer(sentence_input)

d_et=Dropout(0.2)(embedded_sequences)


d_ct= Attention(16,16)([d_et,d_et,d_et])
#l_att=Dense(64,activation='tanh')(l_att)
attention = Dense(200,activation='tanh')(d_ct)
attention = Flatten()(Dense(1)(attention))
attention_weight = Activation('softmax')(attention)
l_att=keras.layers.Dot((1, 1))([d_ct, attention_weight])


sentEncodert = Model([sentence_input], l_att)


review_input = Input((MAX_SENTS,MAX_SENT_LENGTH,), dtype='int32')
review_encoders= TimeDistributed(sentEncodert)(review_input)

reviewaction_input = Input((MAX_SENTS,), dtype='int32')
actionembedding_layer = Embedding(7, 256,trainable=True)
 
action_embedded_sequences = actionembedding_layer(reviewaction_input)

review_encoders = add([review_encoders,action_embedded_sequences])
u_att = Attention(16,16)([review_encoders,review_encoders,review_encoders])


reviewactionmask0 = Input((MAX_SENTS,), dtype='float32')
reviewactionmask1 = Input((MAX_SENTS,), dtype='float32')
reviewactionmask2 = Input((MAX_SENTS,), dtype='float32')
reviewactionmask3 = Input((MAX_SENTS,), dtype='float32')
reviewactionmask4 = Input((MAX_SENTS,), dtype='float32')
reviewactionmask5 = Input((MAX_SENTS,), dtype='float32')
reviewactionmask6 = Input((MAX_SENTS,), dtype='float32')


u_att012raw=Flatten()(Dense(1)(Dense(200,activation='tanh')(review_encoders ))) 

u_att012rawn=Flatten()(Dense(1)(Dense(200,activation='tanh')(review_encoders ))) 

u_att3raw=Flatten()(Dense(1)(Dense(200,activation='tanh')(review_encoders )))
u_att4raw=Flatten()(Dense(1)(Dense(200,activation='tanh')(review_encoders )))
u_att5raw=Flatten()(Dense(1)(Dense(200,activation='tanh')(review_encoders )))
u_att6raw=Flatten()(Dense(1)(Dense(200,activation='tanh')(review_encoders )))

mask012=add([reviewactionmask0,reviewactionmask1,reviewactionmask2])

u_att012=Dot((1, 1))([review_encoders,Activation('softmax')(Lambda (lambda x:x[0]-(1-x[1])*50)([u_att012raw,mask012]))]) 

u_att012n=Dot((1, 1))([review_encoders,Activation('softmax')(Lambda (lambda x:x[0]-(1-x[1])*50)([u_att012rawn,mask012]))])


u_att3=Dot((1, 1))([review_encoders,Activation('softmax')(Lambda (lambda x:x[0]-(1-x[1])*50)([u_att3raw,reviewactionmask3]))])
u_att4=Dot((1, 1))([review_encoders,Activation('softmax')(Lambda (lambda x:x[0]-(1-x[1])*50)([u_att4raw,reviewactionmask4]))])
u_att5=Dot((1, 1))([review_encoders,Activation('softmax')(Lambda (lambda x:x[0]-(1-x[1])*50)([u_att5raw,reviewactionmask5]))])
u_att6=Dot((1, 1))([review_encoders,Activation('softmax')(Lambda (lambda x:x[0]-(1-x[1])*50)([u_att6raw,reviewactionmask6]))])

u_att_strong_exp_pos=Dense(256)(u_att4)
u_att_strong_imp_pos=Dense(256)(u_att3)

posvec =concatenate([Lambda(lambda x: K.expand_dims(x,axis=1))(vec) for vec in [u_att_strong_exp_pos,u_att_strong_imp_pos]],axis=1)
attentionv= Activation('softmax')(Flatten()(Dense(1)(Dense(200,activation='tanh')(posvec))))
u_att_strong_pos=keras.layers.Dot((1, 1))([posvec, attentionv])

negvec =concatenate([Lambda(lambda x: K.expand_dims(x,axis=1))(vec) for vec in [u_att5,u_att6]],axis=1)
attentionv2= Activation('softmax')(Flatten()(Dense(1)(Dense(200,activation='tanh')(negvec))))
u_att_strong_neg=keras.layers.Dot((1, 1))([negvec, attentionv2])


u_att_weak_imp_pos=Dense(256)(u_att012)
u_att_weak_imp_neg=Dense(256)(u_att012n)


#u_att = Dot((1, 1))([review_encoders,Activation('softmax')(Flatten()(Dense(1)(Dense(200,activation='tanh')(review_encoders ))))])
def cosine_distance(vests):
    x, y = vests
    x = K.l2_normalize(x, axis=-1)
    y = K.l2_normalize(y, axis=-1)
    return -K.mean(x * y, axis=-1, keepdims=True)
 
loss1=Lambda(cosine_distance)([u_att_weak_imp_pos,u_att_weak_imp_neg])
loss2=Lambda(cosine_distance)([u_att_weak_imp_pos,u_att_strong_pos])
loss3=Lambda(cosine_distance)([u_att_weak_imp_neg,u_att_strong_neg])

def myloss(y_true, y_pred):
    return K.mean(K.abs(loss1))-K.mean(K.abs(loss2))-K.mean(K.abs(loss3))

candidates=Input((1+npratio,MAX_SENT_LENGTH,), dtype='int32')
candidate_vecs=TimeDistributed(sentEncodert)(candidates)

logits1 = keras.layers.dot([candidate_vecs, u_att_strong_pos], axes=-1)
logits2 = keras.layers.dot([candidate_vecs, u_att_weak_imp_pos], axes=-1)
logits3 = keras.layers.dot([candidate_vecs, u_att_weak_imp_neg], axes=-1)
logits4 = keras.layers.dot([candidate_vecs, u_att_strong_neg], axes=-1)

labeldense=Dense(1)

#pred1dense1=Dense(256)
#pred1dense2=Dense(256)



#logits1 = keras.layers.dot([candidate_vecs, u_att_strong_pos], axes=-1)
#logits2 = keras.layers.dot([candidate_vecs, u_att_weak_imp_pos], axes=-1)
#logits3 = keras.layers.dot([candidate_vecs, u_att_weak_imp_neg], axes=-1)
#logits4 = keras.layers.dot([candidate_vecs, u_att_strong_neg], axes=-1)
#
logitscon = concatenate([Lambda(lambda x: K.expand_dims(x,axis=1))(vec) for vec in [logits1,logits2,logits3,logits4]],axis=1)

logitscon = Activation('softmax')(Flatten()(TimeDistributed(labeldense)(Lambda (lambda x:K.permute_dimensions(x, (0,2,1)))(logitscon))))
candidates_action=Input((1+npratio,), dtype='float32')

model = Model([candidates,review_input,reviewaction_input,reviewactionmask0,reviewactionmask1,
               reviewactionmask2,reviewactionmask3,reviewactionmask4,reviewactionmask5,reviewactionmask6], [logitscon])

def myloss2(y_true, y_pred):
    return K.mean(K.abs(y_true-y_pred)*candidates_action)



model.compile(loss=['categorical_crossentropy'], optimizer=Adam(lr=0.0001),metrics=['acc'])



candidate_one = keras.Input((MAX_SENT_LENGTH,)) 

candidate_one_vec = sentEncodert([candidate_one]) 
#newsEncodert = Model([candidate_one], candidate_one_vec)

score1 = keras.layers.dot([candidate_one_vec, u_att_strong_pos], axes=-1)
score2 = keras.layers.dot([candidate_one_vec, u_att_weak_imp_pos], axes=-1)
score3 = keras.layers.dot([candidate_one_vec, u_att_weak_imp_neg], axes=-1)
score4 = keras.layers.dot([candidate_one_vec, u_att_strong_neg], axes=-1)

scorecon = concatenate([score1,score2,score3,score4])

logitscon = Activation('sigmoid')(labeldense(scorecon))

#score = keras.layers.Activation(keras.activations.sigmoid)(keras.layers.dot([u_att, candidate_one_vec], axes=-1))
model2 = keras.Model([candidate_one,review_input,reviewaction_input,reviewactionmask0,reviewactionmask1,
               reviewactionmask2,reviewactionmask3,reviewactionmask4,reviewactionmask5,reviewactionmask6], logitscon)

from sklearn.metrics import roc_auc_score

In [ ]:
MAX_SENT_LENGTH=30
MAX_SENTS=200
loss_action_weight={0:1.,1:1.,2:1.,3:2.,4:3.,5:3.,6:1.,-1:0.}
def generate_batch_data_random(batch_size):
    idlist = np.arange(len(train_label))
    np.random.shuffle(idlist)
    y=train_label
    batches = [idlist[range(batch_size*i, min(len(y), batch_size*(i+1)))] for i in range(len(y)//batch_size+1)]
    while (True):
        for i in batches:
            item = news_title[train_candidate[i]]
            user = news_title[train_user_his[i]] 
            useraction = train_user_hisaction[i]
            itemaction=np.array([[loss_action_weight[m] for m in t] for t in train_candidate_action[i]])
            useractionmask2=train_user_hismask2[i]

            yield ([item,user,useraction,itemaction,useractionmask2], [y[i]])
            


def generate_batch_data(batch_size):
    idlist = np.arange(len(test_candidate))
    batches = [idlist[range(batch_size*i, min(len(idlist), batch_size*(i+1)))] for i in range(len(idlist)//batch_size+1)]

    while (True):
        for i in batches:
            item = news_title[test_candidate[i]]
            user=news_title[test_user_his[i]]
            useraction = test_user_hisaction[i]
            useractionmask2=test_user_hismask2[i]
            yield ([item,user,useraction,useractionmask2])
            
            
keras.backend.clear_session() 
sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')

embedding_layer = Embedding(len(word_dict1), 300, weights=[lister],trainable=True)
 
embedded_sequences = embedding_layer(sentence_input)

d_et=Dropout(0.2)(embedded_sequences)


d_ct= Attention(16,16)([d_et,d_et,d_et])
#l_att=Dense(64,activation='tanh')(l_att)
attention = Dense(200,activation='tanh')(d_ct)
attention = Flatten()(Dense(1)(attention))
attention_weight = Activation('softmax')(attention)
l_att=keras.layers.Dot((1, 1))([d_ct, attention_weight])


sentEncodert = Model([sentence_input], l_att)


review_input = Input((MAX_SENTS,MAX_SENT_LENGTH,), dtype='int32')
review_encoders= TimeDistributed(sentEncodert)(review_input)

reviewaction_input = Input((MAX_SENTS,), dtype='int32')
actionembedding_layer = Embedding(7, 256,trainable=True)
 
action_embedded_sequences = actionembedding_layer(reviewaction_input)

u_att = add([review_encoders,action_embedded_sequences])
#u_att = Attention(16,16)([review_encoders,review_encoders,review_encoders])


reviewactionmask0 = Input((MAX_SENTS,), dtype='float32')
reviewactionmask1 = Input((MAX_SENTS,), dtype='float32')
reviewactionmask2 = Input((MAX_SENTS,), dtype='float32')

u_att012raw=Flatten()(Dense(1)(Dense(200,activation='tanh')(review_encoders ))) 
 
mask012=reviewactionmask2#add([reviewactionmask0,reviewactionmask1,reviewactionmask2])

u_att012=Dot((1, 1))([review_encoders,Activation('softmax')(Lambda (lambda x:x[0]-(1-x[1])*50)([u_att012raw,mask012]))]) 

candidates=Input((1+npratio,MAX_SENT_LENGTH,), dtype='int32')
candidate_vecs=TimeDistributed(sentEncodert)(candidates)

logits1 = keras.layers.dot([candidate_vecs, u_att012], axes=-1) 
 
logitscon = Activation('softmax')(logits1)
candidates_action=Input((1+npratio,), dtype='float32')

model = Model([candidates,review_input,reviewaction_input,candidates_action,
               reviewactionmask2], [logitscon])


model.compile(loss=['categorical_crossentropy'], optimizer=Adam(lr=0.0001),metrics=['acc'])



candidate_one = keras.Input((MAX_SENT_LENGTH,)) 

candidate_one_vec = sentEncodert([candidate_one]) 
#newsEncodert = Model([candidate_one], candidate_one_vec)

score1 = keras.layers.dot([candidate_one_vec, u_att012], axes=-1)
#score2 = keras.layers.dot([candidate_one_vec, u_att_weak_imp_pos], axes=-1)
#score3 = keras.layers.dot([candidate_one_vec, u_att_weak_imp_neg], axes=-1)
#score4 = keras.layers.dot([candidate_one_vec, u_att_strong_neg], axes=-1)
#
scorecon = score1#concatenate([score1,score2,score3,score4])

logitscon = Activation('sigmoid')(scorecon)

#score = keras.layers.Activation(keras.activations.sigmoid)(keras.layers.dot([u_att, candidate_one_vec], axes=-1))
model2 = keras.Model([candidate_one,review_input,reviewaction_input,
               reviewactionmask2], logitscon)

from sklearn.metrics import roc_auc_score

In [ ]:
def my_auc(label,score):
    label=np.array(label)
    score=np.array(score)
    false_score = score[label==0]
    positive_score = score[label==1]
    num_positive = (label==1).sum()
    num_negative = (label==0).sum()
    positive_score = positive_score.reshape((num_positive,1))
    positive_score = np.repeat(positive_score,num_negative,axis=1)
    false_score = false_score.reshape((1,num_negative))
    false_score = np.repeat(false_score,num_positive,axis=0)
    return 1-((positive_score<false_score).mean()+0.5*(positive_score==false_score).mean())

In [ ]:
from keras.optimizers import *
results=[]
all_pred=[]
for ep in range(5):
    traingen=generate_batch_data_random(50)
    model.fit_generator(traingen, epochs=1,steps_per_epoch=len(train_label)//50)
    valgen=generate_batch_data(50)
    ider=0
    cr = model2.predict_generator(valgen, steps=len(test_label)//50,verbose=1)
    from sklearn.metrics import roc_auc_score
    all_auc=[]
    all_mrr=[]
    all_ndcg=[]
    all_ndcg2=[]
    for m in test_index:
        if np.sum(test_label[m[0]:m[1]])!=0 and m[1]<len(cr):
    
            all_auc.append(roc_auc_score(test_label[m[0]:m[1]],cr[m[0]:m[1],0]))
            all_mrr.append(mrr_score(test_label[m[0]:m[1]],cr[m[0]:m[1],0]))
            all_ndcg.append(ndcg_score(test_label[m[0]:m[1]],cr[m[0]:m[1],0],k=5))
            all_ndcg2.append(ndcg_score(test_label[m[0]:m[1]],cr[m[0]:m[1],0],k=10))
    results.append([np.mean(all_auc),np.mean(all_mrr),np.mean(all_ndcg),np.mean(all_ndcg2)])
    print(np.mean(all_auc),np.mean(all_mrr),np.mean(all_ndcg),np.mean(all_ndcg2))